In [2]:
from pykeen.pipeline import pipeline
from pykeen.models import TransE
from pykeen.datasets import YAGO310
from pykeen.models import predict
from pykeen.training import SLCWATrainingLoop
from pykeen.evaluation import RankBasedEvaluator
from pykeen.regularizers import LpRegularizer
from torch.optim import Adam
import torch
import pandas as pd
from tqdm import tqdm
import pickle
import json

In [3]:
model = torch.load('/nas/home/gujiashe/trans/2022-04-11-20-07-24_af660c1f-7064-4859-b5a7-11bd1b476116/replicates/replicate-00000/trained_model.pkl')
dataset = YAGO310()

KeyboardInterrupt: 

In [ ]:
# # Define evaluator
# evaluator = RankBasedEvaluator(
#     filtered=True,  # Note: this is True by default; we're just being explicit
# )

# # Evaluate your model with not only testing triples,
# # but also filter on validation triples
# results = evaluator.evaluate(
#     model=model,
#     mapped_triples=dataset.testing.mapped_triples,
#     additional_filter_triples=[
#         dataset.training.mapped_triples,
#         dataset.validation.mapped_triples,
#     ],
#     device = "cuda:1",
# )

You're trying to map triples with 18 entities and 0 relations that are not in the training set. These triples will be excluded from the mapping.
In total 18 from 5000 triples were filtered out
You're trying to map triples with 22 entities and 0 relations that are not in the training set. These triples will be excluded from the mapping.
In total 22 from 5000 triples were filtered out
/nas/home/gujiashe/miniconda3/envs/pykeen/lib/python3.10/site-packages/pykeen/nn/representation.py:375: UserWarning: Directly use Embedding.shape instead of num_embeddings.
  warnings.warn(f"Directly use {self.__class__.__name__}.shape instead of num_embeddings.")


Evaluating on cuda:1:   0%|          | 0.00/4.98k [00:00<?, ?triple/s]

In [8]:
# Predict tails
data = pd.read_csv('YAGO3-10/test.txt', sep="\t", header=None)
df = {}
df["head"] = []
df["relation"] = []
df["pred_tail"] = []
df["true_tail"] = []
df["rank"] = []

for head, relation, tail in tqdm(data.to_numpy()):
    if head not in dataset.training.entity_to_id.keys() or tail not in dataset.training.entity_to_id.keys():
        continue
    predicted_tails_df = predict.get_tail_prediction_df(
        model, head, relation, triples_factory=dataset.training,
    )

    pred_tail_row = predicted_tails_df[:1]
    pred_tail = pred_tail_row["tail_label"].item()
    true_tail_row = predicted_tails_df[predicted_tails_df["tail_label"] == tail]
    tail_id = true_tail_row["tail_id"].item()
    rank = predicted_tails_df.index.get_loc(tail_id) + 1
    df["head"].append(head)
    df["relation"].append(relation)
    df["pred_tail"].append(pred_tail)
    df["true_tail"].append(tail)
    df["rank"].append(rank)
    break
df = pd.DataFrame(df)
df


  0%|          | 0/5000 [00:00<?, ?it/s]/nas/home/gujiashe/miniconda3/envs/pykeen/lib/python3.10/site-packages/pykeen/nn/representation.py:375: UserWarning: Directly use Embedding.shape instead of num_embeddings.
  warnings.warn(f"Directly use {self.__class__.__name__}.shape instead of num_embeddings.")
100%|██████████| 5000/5000 [18:21<00:00,  4.54it/s]


,head,relation,pred_tail,true_tail,rank
0,Stan_Collymore,playsFor,Leicester_City_F.C.,England_national_football_team,13
1,Suriname,hasOfficialLanguage,English_language,Dutch_language,6
2,Mantorras,playsFor,Angola_national_football_team,F.C._Alverca,5
3,Pablo_Bonells,isAffiliatedTo,Pumas_Morelos,Club_Celaya,5
4,Jürgen_Habermas,isInterestedIn,Political_philosophy,Political_philosophy,1
...,...,...,...,...,...
4977,Carlo_Cudicini,wasBornIn,Milan,Milan,1
4978,Craig_Hignett,isAffiliatedTo,Middlesbrough_F.C.,Hartlepool_United_F.C.,13
4979,Andrew_Watson_(footballer),wasBornIn,Glasgow,"Georgetown,_Guyana",684
4980,Saša_Radivojević,playsFor,FK_Čukarički,FK_Radnički_Beograd,9


In [9]:
df.to_csv('yago310_top_predictions.tsv', sep="\t")

In [89]:
# dataset = YAGO310()
# dataset.relation_to_id.items()

# dataset = YAGO310()

# model = TransE(triples_factory=dataset.training)
# model.to("cuda")

# optimizer = Adam(params=model.get_grad_params()) 

# training_loop = SLCWATrainingLoop(model=model, triples_factory=dataset.training, optimizer=optimizer) 

# training_loop.train(triples_factory=dataset.training, num_epochs=5, batch_size=256)

# evaluator = RankBasedEvaluator()
# # Get triples to test
# mapped_triples = dataset.testing.mapped_triples
# # Evaluate
# results = evaluator.evaluate(model, mapped_triples, batch_size=64)